In [1]:
%pip install -U openai

     ---------------------------------------- 1.0/1.0 MB 3.8 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 2.7.2
    Uninstalling openai-2.7.2:
      Successfully uninstalled openai-2.7.2
Note: you may need to restart the kernel to use updated packages.


In [6]:
!pip install pulp

     ---------------------------------------- 16.4/16.4 MB 3.8 MB/s eta 0:00:00


## API Key

In [113]:
import os
from getpass import getpass

os.environ["OPENROUTER_API_KEY"] = getpass("请输入您的 OPENROUTER API 密钥: ")
print("OPENROUTER API 密钥设置完成 ✔")

请输入您的 OPENROUTER API 密钥: ········
OPENROUTER API 密钥设置完成 ✔


## System Prompt

In [114]:
react_system_prompt_template = """
你是一名“资源调度 / 运筹规划”智能助手，擅长把复杂的调度问题分解成若干步骤，利用工具查询当前状态、评估方案，并给出一个清晰的调度计划。

【任务目标（根据具体问题自行判断组合）】
- 在多台 Host / 服务器之间分配或迁移 VM / 任务；
- 满足容量、CPU、内存等资源约束，以及硬约束/软约束（如互斥、同机放置、软约束惩罚）；
- 在约束可行的前提下，使目标尽量优秀，例如：
  - 负载尽量均衡；
  - 迁移次数或迁移成本尽量小；
  - 软约束违约的总惩罚尽量低。

你需要将用户给出的调度问题分解为多个步骤。对于每个步骤：
1. 先用 <thought> 说明你此步的目的（例如：分析当前负载、筛选可迁移的 VM、构造候选方案、比较成本等）；
2. 再用可用工具之一给出一个 <action>，从环境/工具中获取信息或执行计算；
3. 根据环境返回的 <observation> 继续思考，逐步收敛到一个“可接受的调度方案”；
4. 当你有足够的信息时，用 <final_answer> 输出最终的调度建议和理由。

所有步骤请严格使用以下 XML 标签格式输出：
- <question> 用户问题（由系统注入）
- <thought> 思考
- <action> 采取的工具操作
- <observation> 工具或环境返回的结果（由系统/工具提供，你不要伪造）
- <final_answer> 最终答案

⸻
【领域模型文档】

在当前项目目录中存在一个 Markdown 文档 `scheduling_model_spec.md`，其中精确定义了：
- Host / VM / 集群的符号与集合；
- 决策变量（x_{h,v}, y_h, u_{c,h}, m_v, s_c, U_h, U^max, U^min 等）；
- 资源容量、集群节点数、互斥约束、迁移计数、负载均衡等约束；
- 综合目标函数：负载均衡 + 迁移开销 + 软约束惩罚。

当你处理“VM–Host 调度 / 迁移 / 再平衡”类问题时，应遵循以下流程：
1. 先通过 <action> 调用 read_file，使用上方文件列表中的绝对路径读取 `scheduling_model_spec.md`；
   例如：<action>read_file("C:/Users/ameth/Desktop/scheduling_model_spec.md")</action>
2. 在随后的 <thought> 中，用自己的话简要总结该模型的关键假设、约束和目标（无需逐字复述）；
3. 之后的推理与方案设计，应尽量保持与该模型一致；如有偏离，需要在 <thought> 或 <final_answer> 中说明理由。

对于与该模型无关的一般性问题，你可以不必读取该文档，按常规调度思路解答即可。

【ILP 工具使用要求（非常重要）】

对于任何“VM–Host 调度 / 迁移 / 再平衡”类型的问题（例如给出具体 VM→Host 分配方案、设计迁移计划等）：

1. 你必须至少调用一次 solve_vm_scheduling_ilp 工具，禁止只凭思考直接给出人工设计的迁移方案。
2. 在第一次输出 <final_answer> 之前，如果尚未调用 solve_vm_scheduling_ilp，则：
   - 你只能输出 <thought> 和 <action>；
   - 严禁进入情况 B（即输出 <final_answer>）。
3. 调用方式示例（任选其一）：
   - 传入 Python dict：
     <action>solve_vm_scheduling_ilp({"hosts": [...], "vms": [...], "initial_placement": {...}, "clusters": [], "weights": {"alpha": 1.0, "beta": 1.0, "gamma": 0.0}})</action>
   - 或传入 JSON 字符串：
     <action>solve_vm_scheduling_ilp("{\"hosts\": [...], \"vms\": [...], \"initial_placement\": {...}, \"clusters\": [], \"weights\": {\"alpha\": 1.0, \"beta\": 1.0, \"gamma\": 0.0}}")</action>
4. 在获得 solve_vm_scheduling_ilp 的 <observation> 之后，你应在 <thought> 中解释：
   - 得到的 assignment、host_util、migration、objective 等字段的含义；
   - 该解如何满足约束、在何种意义下“较优”或“可接受”。
5. 若你认为 ILP 解不理想（例如某些目标没有完全符合直觉），你可以在 <thought> 中基于该解做适度的手动分析，但不得完全忽略 ILP 解，另起炉灶设计一个与其冲突的方案。

⸻
【调度问题的建模提示（供你在 <thought> 里参考）】
- 尝试识别：
  - 实体：Host、VM、集群、机房等；
  - 约束：每台 Host 的容量上限、集群互斥/同机、硬约束 vs 软约束；
  - 目标：负载均衡、迁移成本、软约束惩罚等。
- 可以在思考中显式写出一个简化模型，例如：
  - “将问题视为：在若干 Host 上给 VM 重新分配位置，使最大利用率差异最小，同时迁移 VM 数量尽量少。”
- 当存在硬约束和软约束冲突时：
  - 必须满足硬约束；
  - 对软约束，可以计算或描述“违约代价”，在多个方案之间做权衡。

⸻
【格式与工具调用规则（必须严格遵守）】

1. 你每一轮的回复 **只能** 采用下面两种结构之一，且标签顺序必须严格如下：

   情况 A：需要继续调用工具时，必须输出：
   <thought>这里写你的思考过程</thought>
   <action>这里写要调用的工具及参数，如 read_file("C:/path/file.txt")</action>

   情况 B：已经获得足够信息并给出最终答案时，必须输出：
   <thought>这里写你的思考过程</thought>
   <final_answer>这里写给用户的最终回答</final_answer>

2. 除上述 <thought>、<action>、<final_answer> 标签内容以外，**禁止** 输出任何其他文字、解释、Markdown、列表或额外标签。
   - 不要输出 <think> 或其他自定义标签。
   - 不要输出 JSON、YAML 或代码块格式。
   - 不要在标签外添加任何自然语言。

3. 在同一轮回复中：
   - 必须且只能有一个 <thought> 标签。
   - 要么有且仅有一个 <action> 标签，要么有且仅有一个 <final_answer> 标签。
   - 禁止在同一轮中同时出现 <action> 和 <final_answer>。

4. 输出 <action> 后必须立即停止生成，等待系统提供真实的 <observation>。
   你**不能擅自编造 <observation>**，也不能在没有新 <observation> 的情况下连续调用多个 <action>。

5. 如果你认为已经可以直接回答问题，仍然必须按照情况 B 的格式输出 <thought> 和 <final_answer>，
   不允许只输出自然语言答案或只输出 <final_answer> 而缺少 <thought>。

6. 如果 <action> 中的某个工具参数有多行内容，请使用 \\n 表示，例如：
   <action>write_to_file("C:/tmp/plan.txt", "step1\\nstep2\\nstep3")</action>

7. 工具参数中的文件路径请使用绝对路径，不要只给出文件名。例如：
   <action>write_to_file("C:/tmp/plan.txt", "内容")</action>
   而不是
   <action>write_to_file("plan.txt", "内容")</action>

⸻
【本次任务可用工具（函数签名与说明）】
${tool_list}

⸻
【环境信息】

操作系统：${operating_system}
当前目录下文件列表：${file_list}

请在整个对话中始终保持“运筹调度专家”的视角，清楚说明（在 <thought> 和 <final_answer> 中体现）：
- 调度方案的核心思路；
- 满足了哪些约束；
- 在什么意义下“较优”或“可接受”（例如：牺牲了哪部分软约束来减少迁移成本）。
"""


## Agent

In [115]:
import re
import ast
import inspect
import platform
from string import Template
from typing import List, Callable, Tuple
from openai import OpenAI

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.environ["OPENROUTER_API_KEY"],
)

DEFAULT_MODEL_NAME = "google/gemma-3-27b-it:free" # 选择具体的模型

class ReActAgent:
    def __init__(self, tools: List[Callable], model: str = DEFAULT_MODEL_NAME, project_directory: str = "."):
        # 工具：函数名 -> 函数本体
        self.tools = {func.__name__: func for func in tools}
        self.model_name = model
        self.project_directory = project_directory
        self.client = client
    
    # ========= 对话主循环 =========
    def run(self, user_input: str, max_steps: int = 16):
        messages = [
            {
                "role": "system",
                "content": self.render_system_prompt(react_system_prompt_template),
            },
            {
                "role": "user",
                "content": f"<question>{user_input}</question>",
            },
        ]

        last_content = ""

        for step in range(max_steps):
            print(f"\n===== ReAct step {step + 1} / {max_steps} =====")
            content = self.call_model(messages)
            last_content = content

            # 如果根本没有任何 XML 标签，就直接把整段当最终答案
            if ("<action>" not in content 
                and "<final_answer>" not in content 
                and "<thought>" not in content):
                print("\n[INFO] 本轮模型没有使用 XML 标签，自动将整段内容视作最终答案。")
                return content.strip()

            # 1) Thought（方便看日志）
            thought_match = re.search(r"<thought>(.*?)</thought>", content, re.DOTALL)
            if thought_match:
                thought = thought_match.group(1).strip()
                print(f"\n💭 Thought: {thought}")

            # 2) 先看 final_answer
            final_match = re.search(r"<final_answer>(.*?)</final_answer>", content, re.DOTALL)
            if final_match:
                return final_match.group(1).strip()

            # 3) 再看 action
            action_match = re.search(r"<action>(.*?)</action>", content, re.DOTALL)
            if not action_match:
                print("\n[INFO] 本轮没有 <action>，但已经有内容，作为最终答案返回。")
                return content.strip()

            # 4) 正常 ReAct：解析 action + 调用工具
            action_str = action_match.group(1).strip()
            tool_name, args = self.parse_action(action_str)
            print(f"\n🔧 Action: {tool_name}({', '.join(map(str, args))})")

            if tool_name == "run_terminal_command":
                cont = input("\n是否继续执行终端命令？(Y/N)：")
                if cont.lower() != "y":
                    print("操作已取消。")
                    return "操作被用户取消"

            try:
                observation = self.tools[tool_name](*args)
            except KeyError:
                observation = f"工具 {tool_name} 未定义，请只使用提供的工具列表。"
            except Exception as e:
                observation = f"工具 {tool_name} 执行错误：{e}"

            print(f"\n🔍 Observation：{observation}")
            messages.append({"role": "user", "content": f"<observation>{observation}</observation>"})

        print("\n[WARN] 已达到最大 ReAct 步数，仍未获得 <final_answer>，返回最后一轮模型输出。")
        return last_content.strip()

    # ========= 工具 & 环境描述 =========
    def get_tool_list(self) -> str:
        """生成工具列表字符串，包含函数签名和简要说明"""
        descs = []
        for func in self.tools.values():
            name = func.__name__
            signature = str(inspect.signature(func))
            doc = inspect.getdoc(func) or ""
            descs.append(f"- {name}{signature}: {doc}")
        return "\n".join(descs)

    def render_system_prompt(self, system_prompt_template: str) -> str:
        """用工具列表 & 文件列表渲染 system prompt 模板"""
        tool_list = self.get_tool_list()
        file_list = ", ".join(
            os.path.abspath(os.path.join(self.project_directory, f))
            for f in os.listdir(self.project_directory)
        )
        return Template(system_prompt_template).substitute(
            operating_system=self.get_operating_system_name(),
            tool_list=tool_list,
            file_list=file_list,
        )

    # ========= 调用 OpenRouter 模型 =========
    def call_model(self, messages):
        """调用 OpenRouter 上的聊天模型，并把回复追加到 messages 里"""
        print("\n\n正在请求 OpenRouter 模型……")

        response = self.client.chat.completions.create(
            model=self.model_name,
            messages=messages,
            max_tokens=2048,     # 可选
            temperature=0.2,    # 可选
        )

        content = response.choices[0].message.content
        messages.append({"role": "assistant", "content": content})
        return content

    # ========= 解析 <action> 里的函数调用 =========
    def parse_action(self, code_str: str) -> Tuple[str, List[str]]:
        """
        把形如 'read_file("xxx")' 的字符串解析成 (函数名, [参数列表])
        支持字符串里有括号、逗号、换行等情况。
        """
        match = re.match(r"(\w+)\((.*)\)", code_str, re.DOTALL)
        if not match:
            raise ValueError(f"Invalid function call syntax: {code_str}")

        func_name = match.group(1)
        args_str = match.group(2).strip()

        args = []
        current_arg = ""
        in_string = False
        string_char = None
        paren_depth = 0
        i = 0

        while i < len(args_str):
            char = args_str[i]

            if not in_string:
                if char in ['"', "'"]:
                    in_string = True
                    string_char = char
                    current_arg += char
                elif char == "(":
                    paren_depth += 1
                    current_arg += char
                elif char == ")":
                    paren_depth -= 1
                    current_arg += char
                elif char == "," and paren_depth == 0:
                    # 顶层逗号，结束一个参数
                    args.append(self._parse_single_arg(current_arg.strip()))
                    current_arg = ""
                else:
                    current_arg += char
            else:
                current_arg += char
                # 处理字符串结束（排除转义情况）
                if char == string_char and (i == 0 or args_str[i - 1] != "\\"):
                    in_string = False
                    string_char = None

            i += 1

        if current_arg.strip():
            args.append(self._parse_single_arg(current_arg.strip()))

        return func_name, args

    def _parse_single_arg(self, arg_str: str):
        """解析单个参数成 Python 对象（字符串 / 数字 / 列表 等）"""
        arg_str = arg_str.strip()

        # 字符串字面量
        if (arg_str.startswith('"') and arg_str.endswith('"')) or (
            arg_str.startswith("'") and arg_str.endswith("'")
        ):
            inner = arg_str[1:-1]
            inner = inner.replace('\\"', '"').replace("\\'", "'")
            inner = inner.replace("\\n", "\n").replace("\\t", "\t")
            inner = inner.replace("\\r", "\r").replace("\\\\", "\\")
            return inner

        # 其他类型交给 literal_eval
        try:
            return ast.literal_eval(arg_str)
        except (SyntaxError, ValueError):
            return arg_str

    def get_operating_system_name(self):
        os_map = {"Darwin": "macOS", "Windows": "Windows", "Linux": "Linux"}
        return os_map.get(platform.system(), "Unknown")

## Tools

In [116]:
def read_file(file_path: str) -> str:
    """读取指定文件的全部内容"""
    with open(file_path, "r", encoding="utf-8") as f:
        return f.read()

def write_to_file(file_path: str, content: str) -> str:
    """将内容写入指定文件（支持 \\n 换行转义）"""
    with open(file_path, "w", encoding="utf-8") as f:
        f.write(content.replace("\\n", "\n"))
    return "写入成功"

def run_terminal_command(command: str) -> str:
    """执行终端命令，并返回成功/失败信息"""
    import subprocess
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    return "执行成功" if result.returncode == 0 else result.stderr

import json
import pulp

def solve_vm_scheduling_ilp(spec) -> str:
    """
    使用混合整数规划模型求解 VM–Host 调度问题。

    参数 spec 可以是 Python dict 或 JSON 字符串，字段包括：
    - hosts: [{"id": "H1", "cpu_cap"/"cpu_capacity": 32, "mem_cap"/"mem_capacity": 128, "oc_cpu": 1.0, "oc_mem": 1.0}, ...]
    - vms:   [{"id": "V1", "cpu": 8, "mem": 16, "cluster": null}, ...]
    - initial_placement: {"H1": ["V1","V2"], ...}
    - clusters: [{"id": "C1", "size": 2, "hard_excl": false}, ...]（如无可为空列表）
    - weights: {"alpha": 1.0, "beta": 1.0, "gamma": 0.0}

    返回值为 JSON 字符串，包含 assignment、host_util、migration、objective 等字段。
    """
    # 1）兼容 dict / str 两种调用方式
    if isinstance(spec, str):
        data = json.loads(spec)
    else:
        data = spec
    # ---------------------------------

    hosts = data["hosts"]
    vms = data["vms"]
    clusters = data.get("clusters", [])

    host_ids = [h["id"] for h in hosts]
    vm_ids = [v["id"] for v in vms]
    cluster_ids = [c["id"] for c in clusters]

    # ------- 参数映射（兼容 cpu_cap / mem_cap 命名）-------
    cpu_capacity = {}
    mem_capacity = {}
    oc_cpu = {}
    oc_mem = {}
    for h in hosts:
        hid = h["id"]
        cpu_capacity[hid] = h.get("cpu_capacity", h.get("cpu_cap"))
        mem_capacity[hid] = h.get("mem_capacity", h.get("mem_cap"))
        oc_cpu[hid] = h.get("oc_cpu", 1.0)
        oc_mem[hid] = h.get("oc_mem", 1.0)

    # VM 侧
    vm_cpu = {v["id"]: v["cpu"] for v in vms}
    vm_mem = {v["id"]: v["mem"] for v in vms}
    vm_cluster = {v["id"]: v.get("cluster") for v in vms}

    # ------- 用 initial_placement 推 orig_host（否则全是 None）-------
    orig_host = {v: None for v in vm_ids}
    init_placement = data.get("initial_placement", {})
    for h, vlist in init_placement.items():
        for v in vlist:
            orig_host[v] = h
    # -----------------------------------------------

    # 集群参数
    cluster_size = {c["id"]: c["size"] for c in clusters}
    cluster_hard_excl = {c["id"]: bool(c.get("hard_excl", False)) for c in clusters}

    # 每个集群包含哪些 VM
    cluster_vms = {cid: [] for cid in cluster_ids}
    for v in vm_ids:
        cid = vm_cluster.get(v)
        if cid in cluster_vms:
            cluster_vms[cid].append(v)

    # ------- 目标权重（兼容 alpha / beta / gamma 命名）-------
    weights = data.get("weights", {})
    alpha = float(weights.get("alpha_balance", weights.get("alpha", 1.0)))
    beta  = float(weights.get("beta_migration", weights.get("beta", 1.0)))
    gamma = float(weights.get("gamma_soft", weights.get("gamma", 0.0)))

    # ------- 定义问题 -------
    prob = pulp.LpProblem("VM_Scheduling_Full_Model", pulp.LpMinimize)

    # ------- 决策变量 -------
    # x_{v,h} : VM v 是否放在 Host h 上
    x = pulp.LpVariable.dicts(
        "x", (vm_ids, host_ids), lowBound=0, upBound=1, cat=pulp.LpBinary
    )

    # y_h : Host 是否激活
    y = pulp.LpVariable.dicts(
        "y", host_ids, lowBound=0, upBound=1, cat=pulp.LpBinary
    )

    # u_{c,h} : 集群 c 是否在 Host h 上至少有一个 VM
    if cluster_ids:
        u = pulp.LpVariable.dicts(
            "u", (cluster_ids, host_ids), lowBound=0, upBound=1, cat=pulp.LpBinary
        )
        # s_c : 集群软约束违背程度
        s = pulp.LpVariable.dicts("s", cluster_ids, lowBound=0, cat=pulp.LpContinuous)
    else:
        u = {}
        s = {}

    # m_v : VM 是否迁移
    m = pulp.LpVariable.dicts(
        "m", vm_ids, lowBound=0, upBound=1, cat=pulp.LpBinary
    )

    # U_h, U_max, U_min : CPU 利用率及 max/min
    U = pulp.LpVariable.dicts("U", host_ids, lowBound=0, cat=pulp.LpContinuous)
    U_max = pulp.LpVariable("U_max", lowBound=0, cat=pulp.LpContinuous)
    U_min = pulp.LpVariable("U_min", lowBound=0, cat=pulp.LpContinuous)

    big_M = len(vm_ids)  # Host 激活约束用到：Σ_v x_{v,h} ≤ M · y_h

    # ------- 约束 1：每个 VM 必须恰好在一台 Host 上 -------
    for v in vm_ids:
        prob += (
            pulp.lpSum(x[v][h] for h in host_ids) == 1,
            f"assign_{v}",
        )

    # ------- 约束 2：Host 激活状态 (big-M) -------
    # Σ_v x_{v,h} ≤ M · y_h
    for h in host_ids:
        prob += (
            pulp.lpSum(x[v][h] for v in vm_ids) <= big_M * y[h],
            f"host_active_{h}",
        )

    # ------- 约束 3：CPU / Memory 容量 + 超卖系数 -------
    for h in host_ids:
        # CPU: Σ_v cpu_v x_{v,h} ≤ cpu_h * oc_cpu_h * y_h
        prob += (
            pulp.lpSum(vm_cpu[v] * x[v][h] for v in vm_ids)
            <= cpu_capacity[h] * oc_cpu[h] * y[h],
            f"cpu_cap_{h}",
        )

        # Memory: Σ_v mem_v x_{v,h} ≤ mem_h * oc_mem_h * y_h
        prob += (
            pulp.lpSum(vm_mem[v] * x[v][h] for v in vm_ids)
            <= mem_capacity[h] * oc_mem[h] * y[h],
            f"mem_cap_{h}",
        )

    # ------- 约束 4+5：集群节点数 + 互斥 / 软约束违背 -------
    for cid in cluster_ids:
        vlist = cluster_vms[cid]
        if not vlist:
            # 没有 VM 属于该集群时，直接约束 s_c = 0 即可
            prob += (s[cid] == 0, f"empty_cluster_{cid}")
            continue

        # 线性化 u_{c,h} = min(1, Σ_{v∈V_c} x_{v,h})
        for h in host_ids:
            # 如果有 VM 在该 host 上，则 u_{c,h} 必须为 1
            prob += (
                pulp.lpSum(x[v][h] for v in vlist) >= u[cid][h],
                f"u_lb_{cid}_{h}",
            )
            # 如果 u_{c,h} 为 1，则至少有 1 台 VM 在该 host 上
            prob += (
                pulp.lpSum(x[v][h] for v in vlist)
                <= len(vlist) * u[cid][h],
                f"u_ub_{cid}_{h}",
            )

            # 硬互斥：hard_excl_c = true 时，每台 host 最多该集群 1 台 VM
            if cluster_hard_excl[cid]:
                prob += (
                    pulp.lpSum(x[v][h] for v in vlist) <= 1,
                    f"hard_excl_{cid}_{h}",
                )

        # 节点数 + 软约束违背：
        # Σ_h u_{c,h} + s_c = size_c
        # 若无法把该集群的 VM 分散到指定 size_c 个 host，s_c 会变为 >0，
        # 在目标里作为惩罚项。
        prob += (
            pulp.lpSum(u[cid][h] for h in host_ids) + s[cid]
            == cluster_size[cid],
            f"cluster_size_{cid}",
        )

    # ------- 约束 6：迁移计数 m_v -------
    for v in vm_ids:
        h_orig = orig_host.get(v)
        if h_orig is None:
            # 如果没给出原宿主机，就不计迁移：m_v = 0
            prob += (m[v] == 0, f"migration_none_{v}")
        else:
            # Σ_h x_{v,h} = 1 已经有了
            # 当仍在原宿主机时 x_{v,h_orig} = 1 ⇒ 1 - x_{v,h_orig} = 0 ⇒ m_v ≥ 0
            # 当迁移走时   x_{v,h_orig} = 0 ⇒ 1 - x_{v,h_orig} = 1 ⇒ m_v ≥ 1
            prob += (
                m[v] >= 1 - x[v][h_orig],
                f"migration_lb_{v}",
            )

    # ------- 约束 7：负载均衡 (线性化 max-min) -------
    # 定义 U_h = Σ_v cpu_v x_{v,h} / (cpu_h * oc_cpu_h)
    for h in host_ids:
        denom = cpu_capacity[h] * oc_cpu[h]
        if denom <= 0:
            # 防止奇怪输入导致除零，直接令 U_h = 0
            prob += (U[h] == 0, f"U_zero_{h}")
        else:
            prob += (
                pulp.lpSum(vm_cpu[v] * x[v][h] for v in vm_ids)
                == U[h] * denom,
                f"util_def_{h}",
            )

        # max/min 线性化：U_min ≤ U_h ≤ U_max
        prob += (U[h] <= U_max, f"U_max_{h}")
        prob += (U[h] >= U_min, f"U_min_{h}")

    # ------- 目标函数 -------
    # α (U_max - U_min) + β Σ_v m_v + γ Σ_c s_c
    obj_terms = [
        alpha * (U_max - U_min),
        beta * pulp.lpSum(m[v] for v in vm_ids),
    ]
    if cluster_ids:
        obj_terms.append(gamma * pulp.lpSum(s[cid] for cid in cluster_ids))

    prob += pulp.lpSum(obj_terms), "objective"

    # ------- 求解 -------
    prob.solve(pulp.PULP_CBC_CMD(msg=False))

    status = pulp.LpStatus[prob.status]
    objective = pulp.value(prob.objective)

    # ------- 结果解析 -------
    assignment = []
    for v in vm_ids:
        for h in host_ids:
            if pulp.value(x[v][h]) > 0.5:
                assignment.append({"vm": v, "host": h})
                break

    host_util = {h: float(pulp.value(U[h])) for h in host_ids}
    migration = {v: int(round(pulp.value(m[v]))) for v in vm_ids}

    soft_violation = {}
    if cluster_ids:
        for cid in cluster_ids:
            soft_violation[cid] = float(pulp.value(s[cid]))

    result = {
        "status": status,
        "objective": objective,
        "assignment": assignment,
        "host_util": host_util,
        "U_max": float(pulp.value(U_max)),
        "U_min": float(pulp.value(U_min)),
        "migration": migration,
        "soft_violation": soft_violation,
    }

    return json.dumps(result, ensure_ascii=False, indent=2)

In [117]:
project_dir = "C:/Users/ameth/Desktop"

tools = [read_file, write_to_file, run_terminal_command, solve_vm_scheduling_ilp,]
agent = ReActAgent(
    tools=tools,
    model=DEFAULT_MODEL_NAME,
    project_directory=project_dir,
)

## Test

In [118]:
question = """
当前集群的初始状态如下：

- 有 4 台宿主机 Host-1 ~ Host-4，规格为：
  - H1: cpu_cap = 32, mem_cap = 128
  - H2: cpu_cap = 32, mem_cap = 128
  - H3: cpu_cap = 32, mem_cap = 128
  - H4: cpu_cap = 32, mem_cap = 128

- 有 8 台 VM，初始放置情况为（括号内为所在宿主机）：
  - V1: cpu = 8,  mem = 16, 在 H1
  - V2: cpu = 8,  mem = 16, 在 H1
  - V3: cpu = 8,  mem = 16, 在 H2
  - V4: cpu = 8,  mem = 16, 在 H2
  - V5: cpu = 8,  mem = 16, 在 H3
  - V6: cpu = 8,  mem = 16, 在 H3
  - V7: cpu = 8,  mem = 16, 在 H4
  - V8: cpu = 8,  mem = 16, 在 H4

在这个初始状态下，每台 Host 上各有 2 台 VM，资源利用率比较均衡。

现在系统在某一时刻新增了一台宿主机 Host-5：
- H5: cpu_cap = 32, mem_cap = 128（新增时上面没有 VM）

希望对 VM 做一次 rebalance，把一部分负载迁移到新的 Host-5 上，同时满足以下条件：

【硬约束】
- 每个 VM 必须恰好落在一台 Host 上；
- 每台 Host 上所有 VM 的 CPU / Mem 总和不能超过该 Host 的容量。

【优化目标】
- 各 Host 的 CPU 利用率尽量均衡；
- 发生迁移的 VM 数量（或者迁移成本）尽量少。

请根据这些条件，给出一个合理的迁移 / 重分布方案，包括：
1）每台 Host 最终承载的 VM 列表（H1~H5 各有哪些 VM）；
2）哪些 VM 发生了迁移，从哪台 Host 迁移到了哪台 Host；
3）迁移前后各 Host CPU 利用率的大致变化，以及为什么认为这个方案在“负载均衡”和“迁移次数适度”之间是一个可以接受的折中。
"""

answer = agent.run(question)
print(answer)


===== ReAct step 1 / 16 =====


正在请求 OpenRouter 模型……

💭 Thought: 问题描述了一个 VM 迁移/再平衡的场景，目标是在新增一台 Host 的情况下，重新分配 VM，以达到负载均衡并减少迁移次数。这是一个典型的资源调度问题，可以建模为混合整数规划 (ILP) 来求解。

首先，我需要读取 `scheduling_model_spec.md` 文件，了解该项目的模型定义，包括符号、约束和目标函数。然后，我会将当前集群状态编码为 ILP 的输入，调用 `solve_vm_scheduling_ilp` 工具求解。最后，我会分析 ILP 的输出，解释方案的合理性，并给出详细的迁移计划。

核心思路是：将问题转化为一个优化问题，目标是在满足容量约束的前提下，最小化负载不均衡和迁移成本。

🔧 Action: read_file(C:/Users/ameth/Desktop/scheduling_model_spec.md)

🔍 Observation：# VM–Host 调度优化模型说明（Scheduling Model Spec）

本文档描述一个用于虚拟机（VM）在宿主机（Host）之间重新分布的混合整数规划模型。  
目标是在满足资源与集群约束的前提下，实现**负载均衡**、**尽量减少迁移**，并对**无法满足的互斥约束进行惩罚**。

---

## 1. 集合与索引

- $H$：宿主机集合（物理服务器），允许动态增删  
- $V$：虚拟机集合  
- $C$：集群集合（例如 $C=\{C_1,C_2,C_3,C_4\}$）  
- $V_c \subseteq V$：属于集群 $c$ 的 VM 子集  

索引约定：

- $h \in H$：宿主机索引  
- $v \in V$：虚拟机索引  
- $c \in C$：集群索引  

---

## 2. 参数（Parameters）

### 2.1 资源与集群参数

| 符号 | 含义 |
|------|------|
| $cpu_h$ | Host $h$ 的物理 CPU 容量 |
| $mem_h$ | Host $h$ 的物理内存容量 |
| $oc\_cpu$ | CPU 全局或按 host 的超卖系数（$>1$） |
| $


💭 Thought: 看起来我仍然在传递参数错误。虽然我将所有参数打包成了一个字典，但工具仍然报错说接收到了过多的参数。这表明工具可能期望接收的是 JSON 字符串而不是 Python 字典。

我将尝试将字典转换为 JSON 字符串，然后传递给 `solve_vm_scheduling_ilp` 工具。

🔧 Action: solve_vm_scheduling_ilp({"hosts": [{"id": "H1", "cpu_cap": 32, "mem_cap": 128, "oc_cpu": 1.0, "oc_mem": 1.0}, {"id": "H2", "cpu_cap": 32, "mem_cap": 128, "oc_cpu": 1.0, "oc_mem": 1.0}, {"id": "H3", "cpu_cap": 32, "mem_cap": 128, "oc_cpu": 1.0, "oc_mem": 1.0}, {"id": "H4", "cpu_cap": 32, "mem_cap": 128, "oc_cpu": 1.0, "oc_mem": 1.0}, {"id": "H5", "cpu_cap": 32, "mem_cap": 128, "oc_cpu": 1.0, "oc_mem": 1.0}], "vms": [{"id": "V1", "cpu": 8, "mem": 16, "cluster": null}, {"id": "V2", "cpu": 8, "mem": 16, "cluster": null}, {"id": "V3", "cpu": 8, "mem": 16, "cluster": null}, {"id": "V4", "cpu": 8, "mem": 16, "cluster": null}, {"id": "V5", "cpu": 8, "mem": 16, "cluster": null}, {"id": "V6", "cpu": 8, "mem": 16, "cluster": null}, {"id": "V7", "cpu": 8, "mem": 16, "cluster": null}, {"id": "V8", "cpu": 8, "mem": 16, "cluster": null}]